# Notebook Setup

In [1]:
if 'google.colab' in str(get_ipython()):
  IN_COLLAB = True
else:
  IN_COLLAB = False

#TODO: CHANGE THIS BASED ON YOUR OWN LOCAL SETTINGS
MY_HOME_ABS_PATH = "/Users/jetcalz07/Desktop/MIDS/W210_Capstone/co2-flux-hourly-gpp-modeling"

if IN_COLLAB:
  from google.colab import drive
  drive.mount('/content/drive/')

## Import Modules

In [2]:
# install required modules quietly
required_packages = ['geopandas', 'pyspark', 'azure-storage-blob']

for p in required_packages: 
  try:
      __import__(p)
  except ImportError:
      %pip install {p} --quiet

import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"
os.chdir(MY_HOME_ABS_PATH) # <------------------ ADDED
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Load locale custome modules
import sys
if IN_COLLAB:
  os.chdir(MY_HOME_ABS_PATH)
  sys.path.insert(0,os.path.abspath("./code/src/tools"))
else:
  sys.path.append(os.path.abspath("./code/src/tools"))

from CloudIO.AzStorageClient import AzStorageClient
from data_pipeline_lib import *

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)


# Define paths
root_dir =  MY_HOME_ABS_PATH
tmp_dir =  root_dir + os.sep + '.tmp'
raw_data_dir = root_dir + os.sep + 'data'
data_dir = root_dir + os.sep + 'data/datasets'
cred_dir = root_dir + os.sep + '.cred'
az_cred_file = cred_dir + os.sep + 'azblobcred.json'
if IN_COLLAB:
  raw_data_dir = "/content/drive/MyDrive/CO2_flux_gpp_modeling/DS_capstone_23Spring_CO2/Data"


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# Constant Definitions

In [3]:
# Select monthly features to use
included_features= ['SITE_ID', 'year', 'month', 'TIMESTAMP',
                   'ESACCI-sm',    # ESACCI Soil Moisture (%)
                   'Percent_Snow', # Percentage of snow cover (%)
                   'NDWI',      # Normalized Different Water Index (NDWI)
                   'PET',       # Potential ET (m)
                   'MODIS_PFT', # Plant Function Type
                   'MODIS_LC',  # MODIS Land Cover
                   'Ts',        # Skin temperature (K) ??
                   'LST_Day',   # Daytime land surface temperature (K)
                   'LST_Night', # Nightime land surface temperature (K)
                   'Lai',       # Leaf Area Index (LAI)
                   'Fpar',      # Fraction of photosynthetically active radiation (fPAR)
                   'CSIF-SIFdaily', # All-sky daily average SIF
                   'BESS-PAR',      # Photosynthetic Active Radiation (PAR) (W/m^2)
                   'BESS-PARdiff',  # Diffuse PAR (W/m^2)
                   'BESS-RSDN'      # Shortwave downwelling radiation (W/m^2)
                   ]

# Define in and out files for monthly data
monthly_data_input_fname = data_dir + os.sep + 'data_monthly_v1_0.csv'
monthly_data_output_fname = raw_data_dir + os.sep + "monthly-imputed-v1_2-i.csv"

# Define methods for filling NA (interpolate or -1) and gap-fill (fill or leave be)
impute = True
impute_method = 'knn' # other options are 'interpolate', 'knn', 'constant' or None
resample_monthly = True
knn_imp_cols = ['year', 'month', 'ESACCI-sm', 'Percent_Snow', 'NDWI', 'PET', 'MODIS_LC', 'Ts', 'LST_Day',
                'LST_Night', 'Lai', 'Fpar', 'CSIF-SIFdaily', 'BESS-PAR', 'BESS-PARdiff', 'BESS-RSDN']
k=5
weights='uniform'
c=-1 # if impute_method = 'constant'

### Execute and Save Out

In [4]:
# Execute Monthly Preparation
PrepMonthly = PrepareMonthlyData(included_features, monthly_data_input_fname, tmp_dir)                                
monthly_df_out = PrepMonthly.run(impute, impute_method, resample_monthly, knn_imp_cols, k, weights, c)

Impute method: knn
Resampling and gap filling missing months: True
# sites dropped bc not available in data_dir: 9


234it [00:58,  4.00it/s]


Imputing values where site has 100 percent of feature missing
# of NA features before global impute: 1864
# of NA features after global impute: 0
Confirmed: No NA values remain


In [8]:
# Save out
monthly_df_out.to_csv(monthly_data_output_fname, index=False)